# NN Project, topic #4: Language identification from a text corpus

Students: Shahzaib MUHAMMAD, Adriana NICOARA, Scott TANKARD

## Our group's topic

From: https://arche.univ-lorraine.fr/mod/page/view.php?id=1310565

Project 4: Language identification from a text corpus

Students: Scott TANKARD, Adriana NICOARA, Shahzaib MUHAMMAD

Task: Text classification

Model: Feedforward neural network

Dataset: https://www.kaggle.com/zarajamshaid/language-identification-datasst

Hint: As a preprocessing step, you should transforms the sentences into trigrams at the character level (more details at: https://towardsdatascience.com/deep-neural-network-language-identification-ae1c158f6a7d)

## General project instructions

From: https://arche.univ-lorraine.fr/mod/page/view.php?id=1301340

General project instructions

Project structure

Each project follows the same overall structure:

    Download the data and preprocess it as required for the given task (if needed).
    Write a Dataset class for creating the train and test datasets (and corresponding dataloaders).
    Define the neural network model.
    Define the hyperparameters to create an instance of the model (e.g., hidden space size, number of convolution kernels...) as well as the parameters required to train neural network (e.g., learning rate).
    Write the training loop for training the model.
    Evaluate the model on the test data. In this part, it is expected to choose an appropriate evaluation metric based on your task. For instance, for classification task, accuracy should be computed (but you can also search for 'precision' and 'recall').
    Save the trained model parameters, and the obtained results if needed.


Deliverables

The project should be written using Pytorch (and not Keras/Tensorflow or any other python deep learning framework). You're expected to send a zip file containing:

    A python file implementing steps 1 to 7. This can be either a jupyter notebook or a .py script. A single file is preferred, but you can use several files if that's your style, as long as your main executable script is clearly indicated (for instance it's named 'main.py').
    The obtained results as extra files (e.g., the trained model parameters, a figure with the training loss / validation metric over epochs, a graph comparing different architectures if you want to play arround with it, etc.)

Your code should be commented in order to clarify implementation details, and desciption about the inputs and outputs of functions.

Note: There is no need to send the dataset, since I will download it and run your scripts directly. Therefore, make sure that you do not transform/change the raw data, and if there is some preprocessing involved, include it in your python file(s).


General Hints

    The tasks corresponding to these projects have been extensively studied. Don't hesitate to search online for more information (either tutorials or research papers, even pieces of code if you can adapt it).
    Some datasets are very large. Therefore, you don't need to use all the data, but instead you can extract a subset of it (for instance, only a few languages for language identification, only a few images/classes for image recognition etc.) to have a lighter dataset / model / training procedure.
    Some datasets are provided with a train / test split, but not always. Either way, you can create your own split (a good rule of thumb can be 80% training and 20 % testing).
    It is strongly advised to use validation in order to monitor training (see 'bonus work' in lab 2.2). You can use part of the training data (e.g., 10%) as a validation set.
    It is good practice to start with a light model (very few layers/parameters) and dataset (subset of your whole dataset) for prototyping. The performance won't be very good, but it's useful to check if there are any error in the train/test procedure. Once everything runs smoothly, you can increase the size of the model and use more data.
    If your project includes convolutional or recurrent neural networks, don't wait the corresponding lab: you can already start working on it (you can basically do everything, just using a 'dummy' MLP model instead of a CNN/RNN).

Modifié le: vendredi 21 octobre 2022, 12:46

In [1]:
#CONSTANTS:
filterLanguage=False #Either Filter the dataset or not.

In [ ]:
# STEP 1: Download the data and preprocess it as required for the given task (if needed).
#########

In [2]:
import pandas as pd
data = pd.read_csv('./dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      22000 non-null  object
 1   language  22000 non-null  object
dtypes: object(2)
memory usage: 343.9+ KB


In [3]:
data.head(10)

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
5,エノが行きがかりでバスに乗ってしまい、気分が悪くなった際に助けるが、今すぐバスを降りたいと運...,Japanese
6,tsutinalar i̇ngilizce tsuutina kanadada albert...,Turkish
7,müller mox figura centralis circulorum doctoru...,Latin
8,برقی بار electric charge تمام زیرجوہری ذرات کی...,Urdu
9,シャーリー・フィールドは、サン・ベルナルド・アベニュー沿い市民センターとrtマーティン高校に...,Japanese


In [4]:
unique_lang = data["language"].unique()
print(unique_lang)
print("Number of unique languages: ", len(unique_lang))

['Estonian' 'Swedish' 'Thai' 'Tamil' 'Dutch' 'Japanese' 'Turkish' 'Latin'
 'Urdu' 'Indonesian' 'Portugese' 'French' 'Chinese' 'Korean' 'Hindi'
 'Spanish' 'Pushto' 'Persian' 'Romanian' 'Russian' 'English' 'Arabic']
Number of unique languages:  22


We choose 7 languages to work with:

In [5]:
data_trim = pd.DataFrame(columns=['Text','language'])
lang = None
if(filterLanguage):
    lang = ['Estonian', 'Swedish', 'Indonesian','Turkish','French', 'Romanian', 'English' ]
    data_trim = pd.DataFrame(columns=['Text','language'])
    for l in lang:
        lang_trim = data[data['language'] ==l].sample(1000, random_state = 100)
        data_trim = data_trim.append(lang_trim)
else:
    lang = data['language'].unique()
    data_trim = data

In [6]:
# STEP 1B: Write a Dataset class for creating the train and test datasets.
#########
from sklearn.model_selection import train_test_split
X = data_trim["Text"]
y = data_trim["language"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42,stratify=y)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [158]:
from sklearn.feature_extraction.text import CountVectorizer

def get_trigrams(corpus,n_feat=200):
    """
    Returns a list of the N most common character trigrams from a list of sentences
    params
    ------------
        corpus: list of strings
        n_feat: integer
    """
    #fit the n-gram model
    vectorizer = CountVectorizer(analyzer='char_wb',
                            ngram_range=(3, 3),
                            max_features=n_feat)
    
    X = vectorizer.fit_transform(corpus)
    
    #Get model feature names
    feature_names = vectorizer.get_feature_names()
    return feature_names

In [159]:
features = {}
features_set = set()

for l in lang:
    #get corpus filtered by language
    corpus = train_data[train_data.language==l]['Text']
    #get 200 most frequent trigrams
    trigrams = get_trigrams(corpus)
    
    #add to dict and set
    features[l] = trigrams 
    features_set.update(trigrams)
#create vocabulary list using feature set
vocab = dict()
for i,f in enumerate(features_set):
    vocab[f]=i

In [ ]:
#train count vectoriser using vocabulary
vectorizer = CountVectorizer(analyzer='char_wb',
                             ngram_range=(3, 3),
                             vocabulary=vocab)

#create feature matrix for training set
corpus = train_data['Text']   
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()

train_feat = pd.DataFrame(data=X.toarray(),columns=feature_names)

In [134]:
train_feat

,परि,se,されて,out,تھے,की,يد,میٹ,li,نند,...,мен,și,ना,وند,्र,राज,त्र,sia,하는,กวา
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18697,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
18698,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
#Scale feature matrix 
train_min = train_feat.min()
train_max = train_feat.max()
train_feat = (train_feat - train_min)/(train_max-train_min)

#Add target variable 
train_feat['lang'] = list(train_data['language'])

Matrix data that will be used for training: train_feat

In [137]:
train_feat

,परि,se,されて,out,تھے,की,يد,میٹ,li,نند,...,și,ना,وند,्र,राज,त्र,sia,하는,กวา,lang
0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Persian
1,0.0,0.05,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,French
2,0.0,0.05,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Romanian
3,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Russian
4,0.0,0.00,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,...,0.0,0.066667,0.0,0.142857,0.2,0.0,0.0,0.0,0.0,Hindi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18695,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Pushto
18696,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Russian
18697,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Romanian
18698,0.0,0.10,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Portugese


In [ ]:
features_set

Heatmap representing the number of common trigrams between languages

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#Calculate number of shared trigrams
labels = ['Estonian', 'Swedish', 'Indonesian','Turkish','French', 'Romanian', 'English' ]
mat = []
for i in labels:
    vec = []
    for j in labels:
        l1 = features[i]
        l2 = features[j]
        intersec = [l for l in l1 if l in l2] 
        # print(intersec)
        vec.append(len(intersec))
    mat.append(vec)

print(mat)

#Plot heatmap
lang = ['Estonian', 'Swedish', 'Indonesian','Turkish','French', 'Romanian', 'English' ]
conf_matrix_df = pd.DataFrame(mat,columns=lang,index=lang)

print(conf_matrix_df)
mask = [[ True,  False,  False,  False,  False,  False, False],
       [True,  True,  False,  False,  False,  False, False],
       [True, True,  True,  False,  False,  False, False],
       [True, True, True,  True,  False,  False, False],
       [True, True, True, True,  True,  False, False],
       [True, True, True, True, True, True, False],
       [True, True, True, True, True, True, True]]


plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
# sns.set(font_scale=1.5)
sns.heatmap(conf_matrix_df,cmap='coolwarm',annot=True,fmt='.5g',cbar=False)

plt.savefig('../feat_explore.png',format='png',dpi=150)

In [ ]:
# STEP 2: Write a Dataset class for creating the train and test datasets (and corresponding dataloaders).
#########


In [ ]:
# STEP 3: Define the neural network model.
#########

# https://towardsdatascience.com/deep-neural-network-language-identification-ae1c158f6a7d

try:
    from sklearn.preprocessing import LabelEncoder
    from keras.utils import np_utils

    def fit_encoder():
        #Fit encoder
        encoder = LabelEncoder()
        encoder.fit(['deu', 'eng', 'fra', 'ita', 'por', 'spa'])

    def encode(y):
        """
        Returns a list of one hot encodings
        Params
        ---------
            y: list of language labels
        """

        y_encoded = encoder.transform(y)
        y_dummy = np_utils.to_categorical(y_encoded)

        return y_dummy


    from keras.models import Sequential
    from keras.layers import Dense

    def define_model_keras():
        #Define model
        model = Sequential()
        model.add(Dense(500, input_dim=663, activation='relu'))
        model.add(Dense(500, activation='relu'))
        model.add(Dense(250, activation='relu'))
        model.add(Dense(6, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
except:
    print('keras part not executed')

# loss='categorical_crossentropy',
# optimizer='adam', 
# activation fns: relu x3, softmax x1


# lab3/lab3_mlp_classif.ipynb
# 237:    "However, when training a classification network, we generally use the [Cross Entropy](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) loss function, which alleviates these two issues. This loss is optimized for handling true labels instead of true probabilities per class, so you don't have to worry about it. Besides, it will automatically apply a [Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) non-linearity to the predicted outputs, in order to normalize them as probabilities per class.\n",

# https://adamoudad.github.io/posts/keras_torch_comparison/syntax/

import torch
import torchvision
import torch.nn as nn
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from torch.utils.data import DataLoader, random_split, Subset
import copy

class ClassifModel(nn.Module):
    def __init__(self , input_size , output_size
                 , hidden_size=500
                 , act_fn=nn.ReLU()
                 , n_hidden_layers=1 ):
        super(ClassifModel, self).__init__()

        self.input_layer = nn.Sequential(nn.Linear(input_size, hidden_size), act_fn)

        self.hiddens = nn.ModuleList([])
        for i in range(n_hidden_layers):
            self.hiddens.append( nn.Sequential(nn.Linear(hidden_size, hidden_size), act_fn) )

        print('len of self.hiddens:', len(self.hiddens))

        alt_hidden_size = 250
        # This one is not in the hiddens loop because it has a different out_features size
        self.hidden2 = nn.Sequential( nn.Linear(hidden_size, alt_hidden_size), act_fn )

        # self.output_layer = nn.LazyLinear(output_size)
        self.output_layer = nn.Linear(alt_hidden_size, output_size)

        # NOTE: Unlike keras, we do not need a final softmax fn at the outputs,
        # because it's not necessary with torch.nn.CrossEntropyLoss.
        # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#
        # https://discuss.pytorch.org/t/do-i-need-to-use-softmax-before-nn-crossentropyloss/16739

    def forward(self, x):
        x = self.input_layer(x)

        if len(self.hiddens) != 0:
            for i, l in enumerate(self.hiddens):
                x = self.hiddens[i](x)

        x = self.hidden2(x)
        out = self.output_layer(x)
        return out

INPUT_SIZE = 663
OUTPUT_SIZE = 6
model = ClassifModel(INPUT_SIZE, OUTPUT_SIZE)
model

In [ ]:
# STEP 4: Define the hyperparameters to create an instance of the model 
# (e.g., hidden space size, number of convolution kernels...) 
# as well as the parameters required to train neural network (e.g., learning rate).
#########

# The Conor O'Sullivan article uses adam from Keras with default learning rate,
# and default there is 0.001: https://keras.io/api/optimizers/adam/
learning_rate = 0.001

# lab4/lab4.2_scott.ipynb
# 363:    "    optimizer = torch.optim.Adam(model_tr.parameters(), lr=learning_rate)\n",

optimizer = torch.optim.Adam#(model_tr.parameters(), lr=learning_rate)


In [ ]:
# STEP 5: Write the training loop for training the model.
#########


In [ ]:
# STEP 6: Evaluate the model on the test data. 
# In this part, it is expected to choose an appropriate evaluation metric based on your task. 
# For instance, for classification task, accuracy should be computed (but you can also search for 'precision' and 'recall').
#########


In [ ]:
# STEP 7: Save the trained model parameters, and the obtained results if needed.
#########
